In [1]:
# 1. Connect out notebook with out Google Drive and navigate to the directory in which we saved the data file(s)

from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/E2S/USOpen

Mounted at /content/gdrive
/content/gdrive/MyDrive/E2S/USOpen


In [2]:
# 1. Load what we will need for data wrangling, visualization, and modeling
import numpy as np
import pandas as pd
from datetime import datetime

pd.options.display.max_rows = None
pd.options.display.max_columns = None

# For visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Support functions for much later modeling
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# Classification Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Scoring Functions
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report



In [3]:
transactions_df = pd.read_csv("Trophy All Tips.csv", dtype={ # Use this file overall: 'RHF, Eldar, Will US Open Transaction Level & Aggregated Data - tips.csv'
      'Location': 'category',
      'Reporting Group': 'category',
      'Card Type': 'category',
      'Display Name': 'category',
      'State': 'category',
      'Terminal': 'category',
      'Name': 'str',
      'Employee': 'category'
}, parse_dates=['Created At Time'])
transactions_df.head()

ValueError: Missing column provided to 'parse_dates': 'Created At Time'

In [ ]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168935 entries, 0 to 168934
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Unnamed: 0              168935 non-null  int64         
 1   Location                168935 non-null  category      
 2   Order ID                168935 non-null  int64         
 3   Reporting Group         168935 non-null  category      
 4   Card Type               168935 non-null  category      
 5   Display Name            167298 non-null  category      
 6   Created At Time         168935 non-null  datetime64[ns]
 7   State                   168935 non-null  category      
 8   Terminal                168935 non-null  category      
 9   Name                    168935 non-null  object        
 10  Employee                168935 non-null  category      
 11  Total                   168935 non-null  float64       
 12  Total Credit Card Tips  168935

In [ ]:
for loc in transactions_df["Location"].unique():
  print(loc)

The Garden Grill
The Garden Main Bar
The Garden Market
The Garden Tacos Nachos
Hole 2 7 Beverages
Hole 2 7 Concessions
Hole 2 7 Dog House
Trophy Club Ext Bar
Trophy Club Food Court
Trophy Club Int Bar
Amex Card Member
Hole 1 Concessions
Hole 17 Concessions
Maniac Hill 18th Hole Marra Forni


In [ ]:
transactions_df["Employee"].unique()

['e2sgardengrill', 'e2sgarden2', 'e2sgardenman', 'e2sgardentaco', 'kylereith', ..., 'e2shole17', 'e2shole17man', 'e2smarra', 'marrafornicashier1', 'marrafornimngr']
Length: 35
Categories (35, object): ['adrianberos', 'alexanderson', 'barisugarman', 'courtneyalday', ...,
                          'e2shole17man', 'e2smarra', 'marrafornicashier1', 'marrafornimngr']

# Marra

In [ ]:
transactions_df.loc[transactions_df["Employee"] == "e2smarra"]

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# prompt: save transactions_df.loc[transactions_df["Employee"] == "e2smarra"]  as "marra.csv" file

marra = transactions_df.loc[transactions_df["Location"] == "Maniac Hill 18th Hole Marra Forni"]
marra.to_csv("marra.csv")
marra.shape

(13183, 14)

In [ ]:
marra = marra.drop_duplicates(subset=["Order ID"])
marra.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6996 entries, 155752 to 168933
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              6996 non-null   int64         
 1   Location                6996 non-null   category      
 2   Order ID                6996 non-null   int64         
 3   Reporting Group         6996 non-null   category      
 4   Card Type               6996 non-null   category      
 5   Display Name            6996 non-null   category      
 6   Created At Time         6996 non-null   datetime64[ns]
 7   State                   6996 non-null   category      
 8   Terminal                6996 non-null   category      
 9   Name                    6996 non-null   object        
 10  Employee                6996 non-null   category      
 11  Total                   6996 non-null   float64       
 12  Total Credit Card Tips  6996 non-null   float6

In [ ]:
marra.head(10)

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
155752,1,Maniac Hill 18th Hole Marra Forni,476661018,N/A Bev,Master,Tax,2024-06-10 07:41:40,closed,Maniac Hill 18TH,DEBIT_CARD,e2smarra,4.28,0.00,0.0
155753,2,Maniac Hill 18th Hole Marra Forni,476663061,N/A Bev,Visa,Tax,2024-06-10 08:10:48,closed,Maniac Hill 18TH hole Marra Forni-5,VISA CARDHOLDER,marrafornicashier1,4.71,0.43,0.0
155754,3,Maniac Hill 18th Hole Marra Forni,476661870,Forni Marra,Visa,Tax,2024-06-10 08:11:06,closed,Maniac Hill 18TH,LUIS CAMACHO,marrafornicashier1,23.38,3.05,0.0
155755,4,Maniac Hill 18th Hole Marra Forni,476662564,N/A Bev,Visa,Tax,2024-06-10 08:12:08,closed,Maniac Hill Marra Forni - 1,DANA G SMITH JR,marrafornicashier1,14.12,1.28,0.0
155756,5,Maniac Hill 18th Hole Marra Forni,476675088,Forni Marra,Visa,Tax,2024-06-10 08:14:20,closed,Maniac Hill Marra Forni - 2,DEBIT_CARD,marrafornicashier1,24.07,0.00,0.0
155758,7,Maniac Hill 18th Hole Marra Forni,476663153,Forni Marra,Master,Tax,2024-06-10 08:31:44,closed,Maniac Hill 18TH hole Marra Forni-6,SHELDON NEALY,marrafornicashier1,27.65,2.51,0.0
155760,9,Maniac Hill 18th Hole Marra Forni,476662660,N/A Bev,Amex,Amex Discount,2024-06-10 08:46:32,closed,Maniac Hill Marra Forni - 1,VALUED CUSTOMER,marrafornicashier1,8.47,0.77,-0.8
155762,11,Maniac Hill 18th Hole Marra Forni,476664467,N/A Bev,Visa,Tax,2024-06-10 08:46:42,closed,Maniac Hill Marra Forni - 3,DEBIT_CARD,marrafornicashier1,4.11,0.37,0.0
155763,12,Maniac Hill 18th Hole Marra Forni,476663837,N/A Bev,Visa,Tax,2024-06-10 08:55:31,closed,Maniac Hill 18TH hole Marra Forni-5,VISA CARDHOLDER,marrafornicashier1,4.28,0.00,0.0
155764,13,Maniac Hill 18th Hole Marra Forni,476663868,N/A Bev,Visa,Tax,2024-06-10 08:57:47,closed,Maniac Hill 18TH hole Marra Forni-6,VISA CARDHOLDER,marrafornicashier1,4.28,0.00,0.0


Let's create a table:

Shift | Day 1 | Day 2 | Day 3 | Day 4 | Day 5 | Day 6 | Day 7

AM

PM

In [ ]:
# prompt: create a df with "shift" column and two values "AM" and "PM" strings and 7 other columns named "Day 1", "Day 2", etc. with empty values

import pandas as pd
data = {
    'Day': range(1, 8),
    'AM Shift': [None]*7,
    'PM Shift': [None]*7
}

marra_tips_df = pd.DataFrame(data)

print(marra_tips_df)


   Day AM Shift PM Shift
0    1     None     None
1    2     None     None
2    3     None     None
3    4     None     None
4    5     None     None
5    6     None     None
6    7     None     None


In [ ]:
marra["Created At Time"].dt.hour.head()

155752    7
155753    8
155754    8
155755    8
155756    8
Name: Created At Time, dtype: int32

In [ ]:
marra["day"] = marra["Created At Time"].dt.day - 9
marra["Shift AM/PM"] = marra["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
marra["Shift Overlap"] = (marra["Created At Time"].dt.time >= pd.to_datetime('14:15:00').time()) & (marra["Created At Time"].dt.time <= pd.to_datetime('16:00:00').time())
marra.tail()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts,day,Shift AM/PM,Shift Overlap
168929,2320,Maniac Hill 18th Hole Marra Forni,479037840,Beer,Master,Tax,2024-06-16 17:35:29,closed,Maniac Hill Marra Forni - 2,EMV_CONTACT,e2smarra,23.54,0.0,0.0,7,PM,False
168930,2321,Maniac Hill 18th Hole Marra Forni,479037943,Beer,Master,Tax,2024-06-16 17:35:36,closed,Maniac Hill 18TH,EMV_CONTACT,e2smarra,49.08,2.0,0.0,7,PM,False
168931,2322,Maniac Hill 18th Hole Marra Forni,479037841,Beer,Visa,Tax,2024-06-16 17:36:02,closed,Maniac Hill Marra Forni - 2,RYAN T SCHAPPERT,e2smarra,11.77,0.0,0.0,7,PM,False
168932,2323,Maniac Hill 18th Hole Marra Forni,479037998,Beer,Visa,Tax,2024-06-16 17:36:10,closed,Maniac Hill Marra Forni - 3,VISA CARDHOLDER,e2smarra,23.54,0.0,0.0,7,PM,False
168933,2324,Maniac Hill 18th Hole Marra Forni,479038247,Beer,Amex,Amex Discount,2024-06-16 17:36:21,closed,Maniac Hill 18TH,ASHLIE M CADE,e2smarra,22.19,1.0,-2.2,7,PM,False


In [ ]:
marra_shifts_group = marra.groupby(["day", "Shift AM/PM"])["Total Credit Card Tips"].sum().reset_index(name="tips")
marra_shifts_group.reset_index()

,index,day,Shift AM/PM,tips
0,0,1,AM,259.11
1,1,1,PM,119.39
2,2,2,AM,546.41
3,3,2,PM,205.54
4,4,3,AM,627.21
5,5,3,PM,270.63
6,6,4,AM,1598.72
7,7,4,PM,447.29
8,8,5,AM,1642.05
9,9,5,PM,472.64


In [ ]:
marra_overlap_group = marra.groupby(["day", "Shift Overlap"])["Total Credit Card Tips"].sum().reset_index(name="tips")
marra_overlap_group.loc[marra_overlap_group["Shift Overlap"] == True]

,day,Shift Overlap,tips
1,1,True,63.28
3,2,True,161.13
5,3,True,165.41
7,4,True,217.26
9,5,True,292.30
11,6,True,469.69
13,7,True,338.72


In [ ]:
marra_agg_group = marra.groupby(["day"])["Total Credit Card Tips"].sum().reset_index(name="Total Tips")
marra_agg_group

,day,Total Tips
0,1,378.50
1,2,751.95
2,3,897.84
3,4,2046.01
4,5,2114.69
5,6,2627.86
6,7,1646.90


In [ ]:
marra.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6996 entries, 155752 to 168933
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              6996 non-null   int64         
 1   Location                6996 non-null   category      
 2   Order ID                6996 non-null   int64         
 3   Reporting Group         6996 non-null   category      
 4   Card Type               6996 non-null   category      
 5   Display Name            6996 non-null   category      
 6   Created At Time         6996 non-null   datetime64[ns]
 7   State                   6996 non-null   category      
 8   Terminal                6996 non-null   category      
 9   Name                    6996 non-null   object        
 10  Employee                6996 non-null   category      
 11  Total                   6996 non-null   float64       
 12  Total Credit Card Tips  6996 non-null   float6

# Amex

In [ ]:
# Filter Amex Only:
amex = transactions_df.loc[transactions_df["Location"] == "Amex Card Member"]
amex.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
94422,1,Amex Card Member,476664655,Beer,Amex,Tax,2024-06-10 09:31:16,closed,87253324c0f30e52,VALUED CUSTOMER,e2samexman,37.66,3.42,0.0
94423,2,Amex Card Member,476664655,Food,Amex,Tax,2024-06-10 09:31:16,closed,87253324c0f30e52,VALUED CUSTOMER,e2samexman,37.66,3.42,0.0
94424,3,Amex Card Member,476664655,N/A Bev,Amex,Tax,2024-06-10 09:31:16,closed,87253324c0f30e52,VALUED CUSTOMER,e2samexman,37.66,3.42,0.0
94425,4,Amex Card Member,476664655,Wine,Amex,Tax,2024-06-10 09:31:16,closed,87253324c0f30e52,VALUED CUSTOMER,e2samexman,37.66,3.42,0.0
94426,5,Amex Card Member,476664678,Beer,Amex,Tax,2024-06-10 09:31:38,closed,05817b0e4be48a8d,VALUED CUSTOMER,e2samex,15.30,1.39,0.0


In [ ]:
# prompt: save amex as "amex.csv" file
amex.to_csv("amex.csv")
amex.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19637 entries, 94422 to 114058
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              19637 non-null  int64         
 1   Location                19637 non-null  category      
 2   Order ID                19637 non-null  int64         
 3   Reporting Group         19637 non-null  category      
 4   Card Type               19637 non-null  category      
 5   Display Name            19637 non-null  category      
 6   Created At Time         19637 non-null  datetime64[ns]
 7   State                   19637 non-null  category      
 8   Terminal                19637 non-null  category      
 9   Name                    19637 non-null  object        
 10  Employee                19637 non-null  category      
 11  Total                   19637 non-null  float64       
 12  Total Credit Card Tips  19637 non-null  float6

In [ ]:
# Drop duplicates:
amex = amex.drop_duplicates(subset=["Order ID"])
amex.info()


<class 'pandas.core.frame.DataFrame'>
Index: 6835 entries, 94422 to 114057
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              6835 non-null   int64         
 1   Location                6835 non-null   category      
 2   Order ID                6835 non-null   int64         
 3   Reporting Group         6835 non-null   category      
 4   Card Type               6835 non-null   category      
 5   Display Name            6835 non-null   category      
 6   Created At Time         6835 non-null   datetime64[ns]
 7   State                   6835 non-null   category      
 8   Terminal                6835 non-null   category      
 9   Name                    6835 non-null   object        
 10  Employee                6835 non-null   category      
 11  Total                   6835 non-null   float64       
 12  Total Credit Card Tips  6835 non-null   float64

In [ ]:
# checking duplicates
amex.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
94422,1,Amex Card Member,476664655,Beer,Amex,Tax,2024-06-10 09:31:16,closed,87253324c0f30e52,VALUED CUSTOMER,e2samexman,37.66,3.42,0.0
94426,5,Amex Card Member,476664678,Beer,Amex,Tax,2024-06-10 09:31:38,closed,05817b0e4be48a8d,VALUED CUSTOMER,e2samex,15.30,1.39,0.0
94427,6,Amex Card Member,476664703,Beer,Amex,Tax,2024-06-10 09:31:57,closed,87253324c0f30e52,VALUED CUSTOMER,e2samexman,16.00,2.09,0.0
94428,7,Amex Card Member,476664723,Liquor,Amex,Tax,2024-06-10 09:32:24,closed,05817b0e4be48a8d,VALUED CUSTOMER,e2samex,15.44,1.00,0.0
94429,8,Amex Card Member,476664740,Beer,Amex,Tax,2024-06-10 09:32:40,closed,87253324c0f30e52,VALUED CUSTOMER,e2samexman,31.38,4.09,0.0


In [ ]:
# Make new columns:
amex["day"] = amex["Created At Time"].dt.day - 9
amex["Shift AM/PM"] = amex["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
amex["Shift Overlap"] = (amex["Created At Time"].dt.time >= pd.to_datetime('14:15:00').time()) & (amex["Created At Time"].dt.time <= pd.to_datetime('16:00:00').time())
amex.tail()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts,day,Shift AM/PM,Shift Overlap
114046,3636,Amex Card Member,479033528,Beer,Amex,Amex Discount,2024-06-16 17:19:25,closed,f2fdbd01e3bfc12f,ALLISON JOY BRAGER,e2samex,17.37,1.00,-1.70,7,PM,False
114050,3640,Amex Card Member,479036114,N/A Bev,Amex,Amex Discount,2024-06-16 17:29:44,closed,f2fdbd01e3bfc12f,ASHLEY R FEDERAL,e2samex,8.86,1.16,-0.80,7,PM,False
114052,3642,Amex Card Member,479036954,Beer,Amex,Amex Discount,2024-06-16 17:33:13,closed,f2fdbd01e3bfc12f,VALUED CUSTOMER,e2samex,24.08,0.00,-2.50,7,PM,False
114056,3646,Amex Card Member,479037633,N/A Bev,Visa,Tax,2024-06-16 17:36:08,closed,f2fdbd01e3bfc12f,JEFFREY NANCE,e2samex,9.84,1.28,0.00,7,PM,False
114057,3647,Amex Card Member,479039024,Beer,Amex,Amex Discount,2024-06-16 17:42:07,closed,f2fdbd01e3bfc12f,VALUED CUSTOMER,e2samex,13.24,1.20,-1.25,7,PM,False


In [ ]:
# Making a table for AM/PM shifts:
amex_shifts_group = amex.groupby(["day", "Shift AM/PM"])["Total Credit Card Tips"].sum().reset_index(name="tips")
amex_shifts_group.reset_index()

,index,day,Shift AM/PM,tips
0,0,1,AM,601.11
1,1,1,PM,208.12
2,2,2,AM,663.65
3,3,2,PM,251.66
4,4,3,AM,1151.24
5,5,3,PM,409.90
6,6,4,AM,1735.53
7,7,4,PM,876.62
8,8,5,AM,2042.81
9,9,5,PM,795.14


In [ ]:
# Making a table for Overlap shifts:
amex_overlap_group = amex.groupby(["day", "Shift Overlap"])["Total Credit Card Tips"].sum().reset_index(name="tips")
amex_overlap_group.loc[amex_overlap_group["Shift Overlap"] == True]

,day,Shift Overlap,tips
1,1,True,154.94
3,2,True,191.29
5,3,True,334.61
7,4,True,685.18
9,5,True,557.22
11,6,True,638.96
13,7,True,740.08


In [ ]:
# make table for agg:
amex_agg_group = amex.groupby(["day"])["Total Credit Card Tips"].sum().reset_index(name="Total Tips")
amex_agg_group

,day,Total Tips
0,1,809.23
1,2,915.31
2,3,1561.14
3,4,2612.15
4,5,2837.95
5,6,3472.32
6,7,2409.15


# Hole 1


In [ ]:
# Filter Amex Only:
hole1 = transactions_df.loc[transactions_df["Location"] == "Hole 1 Concessions"]
location = hole1
location.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
114059,1,Hole 1 Concessions,476661048,Food,Visa,Tax,2024-06-10 07:43:53,closed,HOLE 1 CONCESSIONS - 03,VISA CARDHOLDER,healingminds,16.00,2.09,0.0
114060,2,Hole 1 Concessions,476661048,N/A Bev,Visa,Tax,2024-06-10 07:43:53,closed,HOLE 1 CONCESSIONS - 03,VISA CARDHOLDER,healingminds,16.00,2.09,0.0
114061,3,Hole 1 Concessions,476661557,Food,Visa,Tax,2024-06-10 07:52:36,closed,HOLE 1 CONCESSIONS - 03,VISA CARDHOLDER,healingminds,4.71,0.43,0.0
114062,4,Hole 1 Concessions,476662646,N/A Bev,Visa,Tax,2024-06-10 08:14:26,closed,HOLE 1 CONCESSIONS - 03,EMV_CONTACT,healingminds,4.30,0.56,0.0
114063,5,Hole 1 Concessions,476662996,N/A Bev,Visa,Tax,2024-06-10 08:18:44,closed,HOLE 1 CONCESSIONS - 04,SAMUEL G CUNDIFF,healingminds,7.49,0.00,0.0


In [ ]:
# save csv
location.to_csv("hole1.csv")
location.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16995 entries, 114059 to 131053
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              16995 non-null  int64         
 1   Location                16995 non-null  category      
 2   Order ID                16995 non-null  int64         
 3   Reporting Group         16995 non-null  category      
 4   Card Type               16995 non-null  category      
 5   Display Name            16738 non-null  category      
 6   Created At Time         16995 non-null  datetime64[ns]
 7   State                   16995 non-null  category      
 8   Terminal                16995 non-null  category      
 9   Name                    16995 non-null  object        
 10  Employee                16995 non-null  category      
 11  Total                   16995 non-null  float64       
 12  Total Credit Card Tips  16995 non-null  float

In [ ]:
# Drop duplicates:
location = location.drop_duplicates(subset=["Order ID"])
location.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8550 entries, 114059 to 131052
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              8550 non-null   int64         
 1   Location                8550 non-null   category      
 2   Order ID                8550 non-null   int64         
 3   Reporting Group         8550 non-null   category      
 4   Card Type               8550 non-null   category      
 5   Display Name            8293 non-null   category      
 6   Created At Time         8550 non-null   datetime64[ns]
 7   State                   8550 non-null   category      
 8   Terminal                8550 non-null   category      
 9   Name                    8550 non-null   object        
 10  Employee                8550 non-null   category      
 11  Total                   8550 non-null   float64       
 12  Total Credit Card Tips  8550 non-null   float6

In [ ]:
# checking duplicates
location.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
114059,1,Hole 1 Concessions,476661048,Food,Visa,Tax,2024-06-10 07:43:53,closed,HOLE 1 CONCESSIONS - 03,VISA CARDHOLDER,healingminds,16.00,2.09,0.0
114061,3,Hole 1 Concessions,476661557,Food,Visa,Tax,2024-06-10 07:52:36,closed,HOLE 1 CONCESSIONS - 03,VISA CARDHOLDER,healingminds,4.71,0.43,0.0
114062,4,Hole 1 Concessions,476662646,N/A Bev,Visa,Tax,2024-06-10 08:14:26,closed,HOLE 1 CONCESSIONS - 03,EMV_CONTACT,healingminds,4.30,0.56,0.0
114063,5,Hole 1 Concessions,476662996,N/A Bev,Visa,Tax,2024-06-10 08:18:44,closed,HOLE 1 CONCESSIONS - 04,SAMUEL G CUNDIFF,healingminds,7.49,0.00,0.0
114064,6,Hole 1 Concessions,476664112,Beer,Visa,Tax,2024-06-10 08:27:30,closed,HOLE 1 CONCESSIONS - 02,VISA CARDHOLDER,mikeliedl,34.13,3.10,0.0


In [ ]:
# Make new columns:
def new_column(loc):
  loc["day"] = loc["Created At Time"].dt.day - 9
  loc["Shift AM/PM"] = loc["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
  loc["Shift Overlap"] = (loc["Created At Time"].dt.time >= pd.to_datetime('14:15:00').time()) & (loc["Created At Time"].dt.time <= pd.to_datetime('16:00:00').time())
  loc.tail()
  return loc
location = new_column(location)


<ipython-input-86-0af724e54b91>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc["day"] = loc["Created At Time"].dt.day - 9
<ipython-input-86-0af724e54b91>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc["Shift AM/PM"] = loc["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
<ipython-input-86-0af724e54b91>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [ ]:
def tables(loc):
  # Making a table for AM/PM shifts:
  loc_shifts_group = loc.groupby(["day", "Shift AM/PM"])["Total Credit Card Tips"].sum().reset_index(name="tips")
  table1 = loc_shifts_group.reset_index()

  # Making a table for Overlap shifts:
  loc_overlap_group = loc.groupby(["day", "Shift Overlap"])["Total Credit Card Tips"].sum().reset_index(name="tips")

  table2 = loc_overlap_group.loc[loc_overlap_group["Shift Overlap"] == True]

  # make table for agg:
  loc_agg_group = loc.groupby(["day"])["Total Credit Card Tips"].sum().reset_index(name="Total Tips")
  table3 = loc_agg_group
  return table1, table2, table3
table_set = tables(location)


In [ ]:
# Making a table for AM/PM shifts:
table_set[0]

,index,day,Shift AM/PM,tips
0,0,1,AM,534.45
1,1,1,PM,125.72
2,2,2,AM,624.91
3,3,2,PM,118.05
4,4,3,AM,871.19
5,5,3,PM,124.60
6,6,4,AM,1417.25
7,7,4,PM,319.79
8,8,5,AM,1965.96
9,9,5,PM,329.34


In [ ]:
# Making a table for Overlap shifts:
table_set[1]

,day,Shift Overlap,tips
1,1,True,91.26
3,2,True,102.29
5,3,True,119.07
7,4,True,193.18
9,5,True,208.75
11,6,True,710.88
13,7,True,265.87


In [ ]:
# Make table for agg:
table_set[2]

,day,Total Tips
0,1,660.17
1,2,742.96
2,3,995.79
3,4,1737.04
4,5,2295.30
5,6,2640.51
6,7,1494.45


# Hole 17


In [ ]:
# Filter Amex Only:
hole17 = transactions_df.loc[transactions_df["Location"] == "Hole 17 Concessions"]
location = hole17
location.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
131054,1,Hole 17 Concessions,476661206,N/A Bev,Visa,Tax,2024-06-10 07:33:20,closed,HOLE 17 CONCESSIONS - 03,KIMBERLY KETTERING,jaconsinnott,4.41,0.67,0.0
131055,2,Hole 17 Concessions,476662098,N/A Bev,Visa,Tax,2024-06-10 07:38:15,closed,HOLE 17 CONCESSIONS - 01,CAMERON SHOWMAN,e2shole17,3.74,0.00,0.0
131056,3,Hole 17 Concessions,476661830,Food,Amex,Tax,2024-06-10 08:00:11,closed,HOLE 17 CONCESSIONS - 03,MICHAEL GARRITY,jaconsinnott,2.46,0.32,0.0
131057,4,Hole 17 Concessions,476662094,N/A Bev,Visa,Tax,2024-06-10 08:24:17,closed,HOLE 17 CONCESSIONS - 03,JOSEPH W CRISAFULLI,jaconsinnott,7.38,0.96,0.0
131058,5,Hole 17 Concessions,476663502,Beer,Visa,Tax,2024-06-10 08:29:22,closed,Hole 17 - 05,VISA CARDHOLDER,e2shole17,16.05,0.00,0.0


In [ ]:
# save csv
location.to_csv("hole1.csv")
location.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24698 entries, 131054 to 155751
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              24698 non-null  int64         
 1   Location                24698 non-null  category      
 2   Order ID                24698 non-null  int64         
 3   Reporting Group         24698 non-null  category      
 4   Card Type               24698 non-null  category      
 5   Display Name            24491 non-null  category      
 6   Created At Time         24698 non-null  datetime64[ns]
 7   State                   24698 non-null  category      
 8   Terminal                24698 non-null  category      
 9   Name                    24698 non-null  object        
 10  Employee                24698 non-null  category      
 11  Total                   24698 non-null  float64       
 12  Total Credit Card Tips  24698 non-null  float

In [ ]:
# Drop duplicates:
location = location.drop_duplicates(subset=["Order ID"])
location.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11772 entries, 131054 to 155748
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              11772 non-null  int64         
 1   Location                11772 non-null  category      
 2   Order ID                11772 non-null  int64         
 3   Reporting Group         11772 non-null  category      
 4   Card Type               11772 non-null  category      
 5   Display Name            11567 non-null  category      
 6   Created At Time         11772 non-null  datetime64[ns]
 7   State                   11772 non-null  category      
 8   Terminal                11772 non-null  category      
 9   Name                    11772 non-null  object        
 10  Employee                11772 non-null  category      
 11  Total                   11772 non-null  float64       
 12  Total Credit Card Tips  11772 non-null  float

In [ ]:
# checking duplicates
location.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
131054,1,Hole 17 Concessions,476661206,N/A Bev,Visa,Tax,2024-06-10 07:33:20,closed,HOLE 17 CONCESSIONS - 03,KIMBERLY KETTERING,jaconsinnott,4.41,0.67,0.0
131055,2,Hole 17 Concessions,476662098,N/A Bev,Visa,Tax,2024-06-10 07:38:15,closed,HOLE 17 CONCESSIONS - 01,CAMERON SHOWMAN,e2shole17,3.74,0.00,0.0
131056,3,Hole 17 Concessions,476661830,Food,Amex,Tax,2024-06-10 08:00:11,closed,HOLE 17 CONCESSIONS - 03,MICHAEL GARRITY,jaconsinnott,2.46,0.32,0.0
131057,4,Hole 17 Concessions,476662094,N/A Bev,Visa,Tax,2024-06-10 08:24:17,closed,HOLE 17 CONCESSIONS - 03,JOSEPH W CRISAFULLI,jaconsinnott,7.38,0.96,0.0
131058,5,Hole 17 Concessions,476663502,Beer,Visa,Tax,2024-06-10 08:29:22,closed,Hole 17 - 05,VISA CARDHOLDER,e2shole17,16.05,0.00,0.0


In [ ]:
# Make new columns:
def new_column(loc):
  loc["day"] = loc["Created At Time"].dt.day - 9
  loc["Shift AM/PM"] = loc["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
  loc["Shift Overlap"] = (loc["Created At Time"].dt.time >= pd.to_datetime('14:15:00').time()) & (loc["Created At Time"].dt.time <= pd.to_datetime('16:00:00').time())
  loc.tail()
  return loc
location = new_column(location)


<ipython-input-99-0af724e54b91>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc["day"] = loc["Created At Time"].dt.day - 9
<ipython-input-99-0af724e54b91>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc["Shift AM/PM"] = loc["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
<ipython-input-99-0af724e54b91>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [ ]:
def tables(loc):
  # Making a table for AM/PM shifts:
  loc_shifts_group = loc.groupby(["day", "Shift AM/PM"])["Total Credit Card Tips"].sum().reset_index(name="tips")
  table1 = loc_shifts_group.reset_index()

  # Making a table for Overlap shifts:
  loc_overlap_group = loc.groupby(["day", "Shift Overlap"])["Total Credit Card Tips"].sum().reset_index(name="tips")

  table2 = loc_overlap_group.loc[loc_overlap_group["Shift Overlap"] == True]

  # make table for agg:
  loc_agg_group = loc.groupby(["day"])["Total Credit Card Tips"].sum().reset_index(name="Total Tips")
  table3 = loc_agg_group
  return table1, table2, table3
table_set = tables(location)


In [ ]:
# Making a table for AM/PM shifts:
table_set[0]

,index,day,Shift AM/PM,tips
0,0,1,AM,710.19
1,1,1,PM,292.92
2,2,2,AM,1007.61
3,3,2,PM,247.38
4,4,3,AM,1136.94
5,5,3,PM,282.79
6,6,4,AM,2388.66
7,7,4,PM,956.86
8,8,5,AM,2707.56
9,9,5,PM,818.91


In [ ]:
# Making a table for Overlap shifts:
table_set[1]

,day,Shift Overlap,tips
1,1,True,178.92
3,2,True,187.62
5,3,True,225.89
7,4,True,602.67
9,5,True,610.73
11,6,True,1011.67
13,7,True,737.84


In [ ]:
# Make table for agg:
table_set[2]

,day,Total Tips
0,1,1003.11
1,2,1254.99
2,3,1419.73
3,4,3345.52
4,5,3526.47
5,6,3159.74
6,7,3133.75


# Hole 2 7 Beverages





In [ ]:
# Filter Amex Only:
hole2_7_bev = transactions_df.loc[transactions_df["Location"].isin(["Hole 2 7 Beverages"])]
location = hole2_7_bev
location.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
29339,1,Hole 2 7 Beverages,476675282,Beer,Visa,Tax,2024-06-10 11:45:05,closed,916c196999ba275d,VISA CARDHOLDER,e2shole27bev,11.77,0.00,0.0
29340,2,Hole 2 7 Beverages,476676632,N/A Bev,Master,Tax,2024-06-10 11:54:50,closed,e9445fcc52255149,EMV_CONTACT,e2shole27bev,15.30,1.39,0.0
29341,3,Hole 2 7 Beverages,476678972,Beer,Amex,Amex Discount,2024-06-10 12:09:09,closed,916c196999ba275d,BLAKE SMITH,e2shole27bev,12.52,0.00,-1.3
29342,4,Hole 2 7 Beverages,476687155,Beer,Amex,Amex Discount,2024-06-10 12:57:22,closed,0f2b863495dde19a,VALUED CUSTOMER,e2shole27bev,26.49,2.41,-2.5
29343,5,Hole 2 7 Beverages,476692880,N/A Bev,Amex,Tax,2024-06-10 13:33:43,closed,0f2b863495dde19a,EMV_CONTACT,e2shole27bev,18.01,1.64,0.0


In [ ]:
# Drop duplicates:
location = location.drop_duplicates(subset=["Order ID"])
location.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8825 entries, 29339 to 56918
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              8825 non-null   int64         
 1   Location                8825 non-null   category      
 2   Order ID                8825 non-null   int64         
 3   Reporting Group         8825 non-null   category      
 4   Card Type               8825 non-null   category      
 5   Display Name            8825 non-null   category      
 6   Created At Time         8825 non-null   datetime64[ns]
 7   State                   8825 non-null   category      
 8   Terminal                8825 non-null   category      
 9   Name                    8825 non-null   object        
 10  Employee                8825 non-null   category      
 11  Total                   8825 non-null   float64       
 12  Total Credit Card Tips  8825 non-null   float64 

In [ ]:
# checking duplicates
location.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
29339,1,Hole 2 7 Beverages,476675282,Beer,Visa,Tax,2024-06-10 11:45:05,closed,916c196999ba275d,VISA CARDHOLDER,e2shole27bev,11.77,0.00,0.0
29340,2,Hole 2 7 Beverages,476676632,N/A Bev,Master,Tax,2024-06-10 11:54:50,closed,e9445fcc52255149,EMV_CONTACT,e2shole27bev,15.30,1.39,0.0
29341,3,Hole 2 7 Beverages,476678972,Beer,Amex,Amex Discount,2024-06-10 12:09:09,closed,916c196999ba275d,BLAKE SMITH,e2shole27bev,12.52,0.00,-1.3
29342,4,Hole 2 7 Beverages,476687155,Beer,Amex,Amex Discount,2024-06-10 12:57:22,closed,0f2b863495dde19a,VALUED CUSTOMER,e2shole27bev,26.49,2.41,-2.5
29343,5,Hole 2 7 Beverages,476692880,N/A Bev,Amex,Tax,2024-06-10 13:33:43,closed,0f2b863495dde19a,EMV_CONTACT,e2shole27bev,18.01,1.64,0.0


In [ ]:
# Make new columns:
def new_column(loc):
  loc["day"] = loc["Created At Time"].dt.day - 9
  loc["Shift AM/PM"] = loc["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
  loc["Shift Overlap"] = (loc["Created At Time"].dt.time >= pd.to_datetime('14:15:00').time()) & (loc["Created At Time"].dt.time <= pd.to_datetime('16:00:00').time())
  loc.tail()
  return loc
location = new_column(location)


<ipython-input-191-0af724e54b91>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc["day"] = loc["Created At Time"].dt.day - 9
<ipython-input-191-0af724e54b91>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc["Shift AM/PM"] = loc["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
<ipython-input-191-0af724e54b91>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [ ]:
def tables(loc):
  # Making a table for AM/PM shifts:
  loc_shifts_group = loc.groupby(["day", "Shift AM/PM"])["Total Credit Card Tips"].sum().reset_index(name="tips")
  table1 = loc_shifts_group.reset_index()

  # Making a table for Overlap shifts:
  loc_overlap_group = loc.groupby(["day", "Shift Overlap"])["Total Credit Card Tips"].sum().reset_index(name="tips")

  table2 = loc_overlap_group.loc[loc_overlap_group["Shift Overlap"] == True]

  # make table for agg:
  loc_agg_group = loc.groupby(["day"])["Total Credit Card Tips"].sum().reset_index(name="Total Tips")
  table3 = loc_agg_group
  return table1, table2, table3
table_set = tables(location)


In [ ]:
# Making a table for AM/PM shifts:
table_set[0]

,index,day,Shift AM/PM,tips
0,0,1,AM,216.09
1,1,1,PM,79.60
2,2,2,AM,374.76
3,3,2,PM,131.72
4,4,3,AM,597.75
5,5,3,PM,134.42
6,6,4,AM,1935.84
7,7,4,PM,592.43
8,8,5,AM,2539.00
9,9,5,PM,843.07


In [ ]:
# Making a table for Overlap shifts:
table_set[1]

,day,Shift Overlap,tips
1,1,True,79.60
3,2,True,122.96
5,3,True,128.70
7,4,True,470.32
9,5,True,662.42
11,6,True,914.35
13,7,True,693.15


In [ ]:
# Make table for agg:
table_set[2]

,day,Total Tips
0,1,295.69
1,2,506.48
2,3,732.17
3,4,2528.27
4,5,3382.07
5,6,3690.23
6,7,1907.39


# Hole 2 7 Concessions





In [ ]:
# Filter Amex Only:
hole2_7_cons = transactions_df.loc[transactions_df["Location"].isin(["Hole 2 7 Concessions"])]
location = hole2_7_cons
location.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
29751,413,Hole 2 7 Concessions,476667432,Food,Visa,Tax,2024-06-10 10:17:03,closed,HOLE 2 7 CONCESSIONS - 04,SEAN P MCELHENNON,e2shole27,33.83,4.41,0.0
29752,414,Hole 2 7 Concessions,476668640,Food,Visa,Tax,2024-06-10 10:33:14,closed,HOLE 2 7 CONCESSIONS - 04,VISA CARDHOLDER,e2shole27,2.14,0.00,0.0
29753,415,Hole 2 7 Concessions,476670101,Food,Amex,Tax,2024-06-10 10:44:41,closed,Hole 2 7 Concessions - 1,VALUED CUSTOMER,e2shole27,6.74,0.00,0.0
29754,416,Hole 2 7 Concessions,476667732,N/A Bev,Visa,Tax,2024-06-10 09:54:12,closed,Hole 2 7 Concessions - 1,DEBIT_CARD,e2shole27,8.56,0.00,0.0
29755,417,Hole 2 7 Concessions,476661453,N/A Bev,Visa,Tax,2024-06-10 07:39:37,closed,HOLE 2 7 CONCESSIONS - 04,VISA CARDHOLDER,adrianberos,13.54,1.23,0.0


In [ ]:
# Drop duplicates:
location = location.drop_duplicates(subset=["Order ID"])
location.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5829 entries, 29751 to 59421
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              5829 non-null   int64         
 1   Location                5829 non-null   category      
 2   Order ID                5829 non-null   int64         
 3   Reporting Group         5829 non-null   category      
 4   Card Type               5829 non-null   category      
 5   Display Name            4975 non-null   category      
 6   Created At Time         5829 non-null   datetime64[ns]
 7   State                   5829 non-null   category      
 8   Terminal                5829 non-null   category      
 9   Name                    5829 non-null   object        
 10  Employee                5829 non-null   category      
 11  Total                   5829 non-null   float64       
 12  Total Credit Card Tips  5829 non-null   float64 

In [ ]:
# checking duplicates
location.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
29751,413,Hole 2 7 Concessions,476667432,Food,Visa,Tax,2024-06-10 10:17:03,closed,HOLE 2 7 CONCESSIONS - 04,SEAN P MCELHENNON,e2shole27,33.83,4.41,0.0
29752,414,Hole 2 7 Concessions,476668640,Food,Visa,Tax,2024-06-10 10:33:14,closed,HOLE 2 7 CONCESSIONS - 04,VISA CARDHOLDER,e2shole27,2.14,0.00,0.0
29753,415,Hole 2 7 Concessions,476670101,Food,Amex,Tax,2024-06-10 10:44:41,closed,Hole 2 7 Concessions - 1,VALUED CUSTOMER,e2shole27,6.74,0.00,0.0
29754,416,Hole 2 7 Concessions,476667732,N/A Bev,Visa,Tax,2024-06-10 09:54:12,closed,Hole 2 7 Concessions - 1,DEBIT_CARD,e2shole27,8.56,0.00,0.0
29755,417,Hole 2 7 Concessions,476661453,N/A Bev,Visa,Tax,2024-06-10 07:39:37,closed,HOLE 2 7 CONCESSIONS - 04,VISA CARDHOLDER,adrianberos,13.54,1.23,0.0


In [ ]:
# Make new columns:
def new_column(loc):
  loc["day"] = loc["Created At Time"].dt.day - 9
  loc["Shift AM/PM"] = loc["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
  loc["Shift Overlap"] = (loc["Created At Time"].dt.time >= pd.to_datetime('14:15:00').time()) & (loc["Created At Time"].dt.time <= pd.to_datetime('16:00:00').time())
  loc.tail()
  return loc
location = new_column(location)


<ipython-input-123-0af724e54b91>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc["day"] = loc["Created At Time"].dt.day - 9
<ipython-input-123-0af724e54b91>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc["Shift AM/PM"] = loc["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
<ipython-input-123-0af724e54b91>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [ ]:
def tables(loc):
  # Making a table for AM/PM shifts:
  loc_shifts_group = loc.groupby(["day", "Shift AM/PM"])["Total Credit Card Tips"].sum().reset_index(name="tips")
  table1 = loc_shifts_group.reset_index()

  # Making a table for Overlap shifts:
  loc_overlap_group = loc.groupby(["day", "Shift Overlap"])["Total Credit Card Tips"].sum().reset_index(name="tips")

  table2 = loc_overlap_group.loc[loc_overlap_group["Shift Overlap"] == True]

  # make table for agg:
  loc_agg_group = loc.groupby(["day"])["Total Credit Card Tips"].sum().reset_index(name="Total Tips")
  table3 = loc_agg_group
  return table1, table2, table3
table_set = tables(location)


In [ ]:
# Making a table for AM/PM shifts:
table_set[0]

,index,day,Shift AM/PM,tips
0,0,1,AM,674.26
1,1,1,PM,175.96
2,2,2,AM,1026.37
3,3,2,PM,129.92
4,4,3,AM,1582.26
5,5,3,PM,175.38
6,6,4,AM,1090.57
7,7,5,AM,459.85
8,8,6,AM,60.78
9,9,7,AM,1403.63


In [ ]:
# Making a table for Overlap shifts:
table_set[1]

,day,Shift Overlap,tips
1,1,True,102.38
3,2,True,94.95
5,3,True,147.53


In [ ]:
# Make table for agg:
table_set[2]

,day,Total Tips
0,1,850.22
1,2,1156.29
2,3,1757.64
3,4,1090.57
4,5,459.85
5,6,60.78
6,7,1403.63


# Hole 2 7 Dog House





In [ ]:
# Filter Amex Only:
hole2_7_dog = transactions_df.loc[transactions_df["Location"].isin(["Hole 2 7 Dog House"])]
location = hole2_7_dog
location.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
31068,1730,Hole 2 7 Dog House,476668422,N/A Bev,Visa,Tax,2024-06-10 10:24:47,closed,Hole 2 7 Doghouse - 6,VISA CARDHOLDER,e2shole27dog,4.92,0.64,0.0
31069,1731,Hole 2 7 Dog House,476669343,Food,Amex,Tax,2024-06-10 10:37:26,closed,Hole 2 7 Doghouse - 6,ARNOLD E THREATT,e2shole27dog,27.02,2.46,0.0
31070,1732,Hole 2 7 Dog House,476671814,Beer,Amex,Tax,2024-06-10 10:43:34,closed,Hole 2 7 Doghouse - 6,VALUED CUSTOMER,e2shole27dog,18.01,1.64,0.0
31071,1733,Hole 2 7 Dog House,476670529,Food,Discover,Tax,2024-06-10 10:58:11,closed,Hole 2 7 Doghouse - 4,DISCOVER CARDMEMBER,e2shole27dog,15.30,1.39,0.0
31072,1734,Hole 2 7 Dog House,476670743,N/A Bev,Amex,Tax,2024-06-10 11:00:10,closed,Hole 2 7 Doghouse - 4,VALUED CUSTOMER,e2shole27dog,13.77,1.25,0.0


In [ ]:
# Drop duplicates:
location = location.drop_duplicates(subset=["Order ID"])
location.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1319 entries, 31068 to 39421
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              1319 non-null   int64         
 1   Location                1319 non-null   category      
 2   Order ID                1319 non-null   int64         
 3   Reporting Group         1319 non-null   category      
 4   Card Type               1319 non-null   category      
 5   Display Name            1319 non-null   category      
 6   Created At Time         1319 non-null   datetime64[ns]
 7   State                   1319 non-null   category      
 8   Terminal                1319 non-null   category      
 9   Name                    1319 non-null   object        
 10  Employee                1319 non-null   category      
 11  Total                   1319 non-null   float64       
 12  Total Credit Card Tips  1319 non-null   float64 

In [ ]:
# checking duplicates
location.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
31068,1730,Hole 2 7 Dog House,476668422,N/A Bev,Visa,Tax,2024-06-10 10:24:47,closed,Hole 2 7 Doghouse - 6,VISA CARDHOLDER,e2shole27dog,4.92,0.64,0.0
31069,1731,Hole 2 7 Dog House,476669343,Food,Amex,Tax,2024-06-10 10:37:26,closed,Hole 2 7 Doghouse - 6,ARNOLD E THREATT,e2shole27dog,27.02,2.46,0.0
31070,1732,Hole 2 7 Dog House,476671814,Beer,Amex,Tax,2024-06-10 10:43:34,closed,Hole 2 7 Doghouse - 6,VALUED CUSTOMER,e2shole27dog,18.01,1.64,0.0
31071,1733,Hole 2 7 Dog House,476670529,Food,Discover,Tax,2024-06-10 10:58:11,closed,Hole 2 7 Doghouse - 4,DISCOVER CARDMEMBER,e2shole27dog,15.30,1.39,0.0
31072,1734,Hole 2 7 Dog House,476670743,N/A Bev,Amex,Tax,2024-06-10 11:00:10,closed,Hole 2 7 Doghouse - 4,VALUED CUSTOMER,e2shole27dog,13.77,1.25,0.0


In [ ]:
# Make new columns:
def new_column(loc):
  loc["day"] = loc["Created At Time"].dt.day - 9
  loc["Shift AM/PM"] = loc["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
  loc["Shift Overlap"] = (loc["Created At Time"].dt.time >= pd.to_datetime('14:15:00').time()) & (loc["Created At Time"].dt.time <= pd.to_datetime('16:00:00').time())
  loc.tail()
  return loc
location = new_column(location)


<ipython-input-131-0af724e54b91>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc["day"] = loc["Created At Time"].dt.day - 9
<ipython-input-131-0af724e54b91>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc["Shift AM/PM"] = loc["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
<ipython-input-131-0af724e54b91>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [ ]:
def tables(loc):
  # Making a table for AM/PM shifts:
  loc_shifts_group = loc.groupby(["day", "Shift AM/PM"])["Total Credit Card Tips"].sum().reset_index(name="tips")
  table1 = loc_shifts_group.reset_index()

  # Making a table for Overlap shifts:
  loc_overlap_group = loc.groupby(["day", "Shift Overlap"])["Total Credit Card Tips"].sum().reset_index(name="tips")

  table2 = loc_overlap_group.loc[loc_overlap_group["Shift Overlap"] == True]

  # make table for agg:
  loc_agg_group = loc.groupby(["day"])["Total Credit Card Tips"].sum().reset_index(name="Total Tips")
  table3 = loc_agg_group
  return table1, table2, table3
table_set = tables(location)


In [ ]:
# Making a table for AM/PM shifts:
table_set[0]

,index,day,Shift AM/PM,tips
0,0,1,AM,326.74
1,1,1,PM,70.47
2,2,2,AM,376.28
3,3,2,PM,35.37
4,4,3,AM,620.46
5,5,3,PM,116.57


In [ ]:
# Making a table for Overlap shifts:
table_set[1]

,day,Shift Overlap,tips
1,1,True,58.70
3,2,True,33.07
5,3,True,113.39


In [ ]:
# Make table for agg:
table_set[2]

,day,Total Tips
0,1,397.21
1,2,411.65
2,3,737.03


# The Garden Grill





In [ ]:
# Filter Amex Only:
garden_grill = transactions_df.loc[transactions_df["Location"].isin(["The Garden Grill"])]
location = garden_grill
location.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
0,1,The Garden Grill,476689310,N/A Bev,Visa,Tax,2024-06-10 12:50:40,closed,Garden concessions-3,VISA CARDHOLDER,e2sgardengrill,59.38,0.00,0.0
1,2,The Garden Grill,476692366,Food,Visa,Tax,2024-06-10 13:13:28,closed,Garden Grill concessions-2,VISA CARDHOLDER,e2sgardengrill,21.87,1.00,0.0
2,3,The Garden Grill,476694005,Food,Amex,Tax,2024-06-10 13:40:10,closed,Garden concessions-3,JULIE HASSETT,e2sgardengrill,15.50,2.02,0.0
3,4,The Garden Grill,476695365,N/A Bev,Amex,Amex Discount,2024-06-10 13:49:08,closed,Garden Grill concessions-2,SARA VELARDE,e2sgardengrill,4.24,0.39,-0.4
4,5,The Garden Grill,476664819,Food,Visa,Tax,2024-06-10 09:27:54,closed,Garden Grill concessions-5,VISA CARDHOLDER,e2sgardengrill,9.63,0.00,0.0


In [ ]:
# Drop duplicates:
location = location.drop_duplicates(subset=["Order ID"])
location.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7893 entries, 0 to 26978
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              7893 non-null   int64         
 1   Location                7893 non-null   category      
 2   Order ID                7893 non-null   int64         
 3   Reporting Group         7893 non-null   category      
 4   Card Type               7893 non-null   category      
 5   Display Name            7702 non-null   category      
 6   Created At Time         7893 non-null   datetime64[ns]
 7   State                   7893 non-null   category      
 8   Terminal                7893 non-null   category      
 9   Name                    7893 non-null   object        
 10  Employee                7893 non-null   category      
 11  Total                   7893 non-null   float64       
 12  Total Credit Card Tips  7893 non-null   float64     

In [ ]:
# checking duplicates
location.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
0,1,The Garden Grill,476689310,N/A Bev,Visa,Tax,2024-06-10 12:50:40,closed,Garden concessions-3,VISA CARDHOLDER,e2sgardengrill,59.38,0.00,0.0
1,2,The Garden Grill,476692366,Food,Visa,Tax,2024-06-10 13:13:28,closed,Garden Grill concessions-2,VISA CARDHOLDER,e2sgardengrill,21.87,1.00,0.0
2,3,The Garden Grill,476694005,Food,Amex,Tax,2024-06-10 13:40:10,closed,Garden concessions-3,JULIE HASSETT,e2sgardengrill,15.50,2.02,0.0
3,4,The Garden Grill,476695365,N/A Bev,Amex,Amex Discount,2024-06-10 13:49:08,closed,Garden Grill concessions-2,SARA VELARDE,e2sgardengrill,4.24,0.39,-0.4
4,5,The Garden Grill,476664819,Food,Visa,Tax,2024-06-10 09:27:54,closed,Garden Grill concessions-5,VISA CARDHOLDER,e2sgardengrill,9.63,0.00,0.0


In [ ]:
# Make new columns:
def new_column(loc):
  loc["day"] = loc["Created At Time"].dt.day - 9
  loc["Shift AM/PM"] = loc["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
  loc["Shift Overlap"] = (loc["Created At Time"].dt.time >= pd.to_datetime('14:15:00').time()) & (loc["Created At Time"].dt.time <= pd.to_datetime('16:00:00').time())
  loc.tail()
  return loc
location = new_column(location)


<ipython-input-139-0af724e54b91>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc["day"] = loc["Created At Time"].dt.day - 9
<ipython-input-139-0af724e54b91>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc["Shift AM/PM"] = loc["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
<ipython-input-139-0af724e54b91>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [ ]:
def tables(loc):
  # Making a table for AM/PM shifts:
  loc_shifts_group = loc.groupby(["day", "Shift AM/PM"])["Total Credit Card Tips"].sum().reset_index(name="tips")
  table1 = loc_shifts_group.reset_index()

  # Making a table for Overlap shifts:
  loc_overlap_group = loc.groupby(["day", "Shift Overlap"])["Total Credit Card Tips"].sum().reset_index(name="tips")

  table2 = loc_overlap_group.loc[loc_overlap_group["Shift Overlap"] == True]

  # make table for agg:
  loc_agg_group = loc.groupby(["day"])["Total Credit Card Tips"].sum().reset_index(name="Total Tips")
  table3 = loc_agg_group
  return table1, table2, table3
table_set = tables(location)


In [ ]:
# Making a table for AM/PM shifts:
table_set[0]

,index,day,Shift AM/PM,tips
0,0,1,AM,538.78
1,1,1,PM,65.15
2,2,2,AM,745.18
3,3,2,PM,75.70
4,4,3,AM,927.95
5,5,3,PM,95.96
6,6,4,AM,2336.01
7,7,4,PM,395.27
8,8,5,AM,2706.71
9,9,5,PM,469.08


In [ ]:
# Making a table for Overlap shifts:
table_set[1]

,day,Shift Overlap,tips
1,1,True,41.69
3,2,True,53.56
5,3,True,76.86
7,4,True,281.58
9,5,True,330.59
11,6,True,381.66
13,7,True,368.86


In [ ]:
# Make table for agg:
table_set[2]

,day,Total Tips
0,1,603.93
1,2,820.88
2,3,1023.91
3,4,2731.28
4,5,3175.79
5,6,2942.27
6,7,1759.13


# The Garden Main Bar





In [ ]:
# Filter Amex Only:
garden_grill_main = transactions_df.loc[transactions_df["Location"].isin(["The Garden Main Bar"])]
location = garden_grill_main
location.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
702,703,The Garden Main Bar,476699207,N/A Bev,Visa,Tax,2024-06-10 14:21:51,closed,d7cc7b1503fa5c90,VISA CARDHOLDER,e2sgarden2,26.08,2.00,0.0
703,704,The Garden Main Bar,476700703,Beer,Visa,Tax,2024-06-10 14:34:20,closed,fdb4f93f990686b2,VISA CARDHOLDER,e2sgarden2,13.91,0.00,0.0
704,705,The Garden Main Bar,476702496,Beer,Amex,Tax,2024-06-10 14:47:41,closed,3d49082bea440958,VALUED CUSTOMER,e2sgarden2,32.20,4.91,0.0
705,706,The Garden Main Bar,476708216,Beer,Amex,Tax,2024-06-10 15:31:08,closed,f41f66e7078a2583,VALUED CUSTOMER,e2sgarden2,42.36,3.85,0.0
706,707,The Garden Main Bar,476710696,Beer,Amex,Tax,2024-06-10 15:49:53,closed,f41f66e7078a2583,VALUED CUSTOMER,e2sgarden2,39.78,6.07,0.0


In [ ]:
# Drop duplicates:
location = location.drop_duplicates(subset=["Order ID"])
location.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5728 entries, 702 to 29335
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              5728 non-null   int64         
 1   Location                5728 non-null   category      
 2   Order ID                5728 non-null   int64         
 3   Reporting Group         5728 non-null   category      
 4   Card Type               5728 non-null   category      
 5   Display Name            5728 non-null   category      
 6   Created At Time         5728 non-null   datetime64[ns]
 7   State                   5728 non-null   category      
 8   Terminal                5728 non-null   category      
 9   Name                    5728 non-null   object        
 10  Employee                5728 non-null   category      
 11  Total                   5728 non-null   float64       
 12  Total Credit Card Tips  5728 non-null   float64   

In [ ]:
# checking duplicates
location.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
702,703,The Garden Main Bar,476699207,N/A Bev,Visa,Tax,2024-06-10 14:21:51,closed,d7cc7b1503fa5c90,VISA CARDHOLDER,e2sgarden2,26.08,2.00,0.0
703,704,The Garden Main Bar,476700703,Beer,Visa,Tax,2024-06-10 14:34:20,closed,fdb4f93f990686b2,VISA CARDHOLDER,e2sgarden2,13.91,0.00,0.0
704,705,The Garden Main Bar,476702496,Beer,Amex,Tax,2024-06-10 14:47:41,closed,3d49082bea440958,VALUED CUSTOMER,e2sgarden2,32.20,4.91,0.0
705,706,The Garden Main Bar,476708216,Beer,Amex,Tax,2024-06-10 15:31:08,closed,f41f66e7078a2583,VALUED CUSTOMER,e2sgarden2,42.36,3.85,0.0
706,707,The Garden Main Bar,476710696,Beer,Amex,Tax,2024-06-10 15:49:53,closed,f41f66e7078a2583,VALUED CUSTOMER,e2sgarden2,39.78,6.07,0.0


In [ ]:
# Make new columns:
def new_column(loc):
  loc["day"] = loc["Created At Time"].dt.day - 9
  loc["Shift AM/PM"] = loc["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
  loc["Shift Overlap"] = (loc["Created At Time"].dt.time >= pd.to_datetime('14:15:00').time()) & (loc["Created At Time"].dt.time <= pd.to_datetime('16:00:00').time())
  loc.tail()
  return loc
location = new_column(location)


<ipython-input-147-0af724e54b91>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc["day"] = loc["Created At Time"].dt.day - 9
<ipython-input-147-0af724e54b91>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc["Shift AM/PM"] = loc["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
<ipython-input-147-0af724e54b91>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [ ]:
def tables(loc):
  # Making a table for AM/PM shifts:
  loc_shifts_group = loc.groupby(["day", "Shift AM/PM"])["Total Credit Card Tips"].sum().reset_index(name="tips")
  table1 = loc_shifts_group.reset_index()

  # Making a table for Overlap shifts:
  loc_overlap_group = loc.groupby(["day", "Shift Overlap"])["Total Credit Card Tips"].sum().reset_index(name="tips")

  table2 = loc_overlap_group.loc[loc_overlap_group["Shift Overlap"] == True]

  # make table for agg:
  loc_agg_group = loc.groupby(["day"])["Total Credit Card Tips"].sum().reset_index(name="Total Tips")
  table3 = loc_agg_group
  return table1, table2, table3
table_set = tables(location)


In [ ]:
# Making a table for AM/PM shifts:
table_set[0]

,index,day,Shift AM/PM,tips
0,0,1,AM,844.83
1,1,1,PM,428.64
2,2,2,AM,1475.53
3,3,2,PM,710.38
4,4,3,AM,2253.61
5,5,3,PM,735.96
6,6,4,AM,2657.29
7,7,5,AM,1377.88
8,8,6,AM,1119.63
9,9,7,AM,2588.49


In [ ]:
# Making a table for Overlap shifts:
table_set[1]

,day,Shift Overlap,tips
1,1,True,259.39
3,2,True,461.95
5,3,True,544.79
10,7,True,192.37


In [ ]:
# Make table for agg:
table_set[2]

,day,Total Tips
0,1,1273.47
1,2,2185.91
2,3,2989.57
3,4,2657.29
4,5,1377.88
5,6,1119.63
6,7,2780.86


# The Garden Market





In [ ]:
# Filter Amex Only:
garden_grill_market = transactions_df.loc[transactions_df["Location"].isin(["The Garden Market"])]
location = garden_grill_market
location.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
1486,1487,The Garden Market,476705704,N/A Bev,Visa,Tax,2024-06-10 15:12:30,closed,df402438d0ea20b5,VISA CARDHOLDER,e2sgardenman,30.01,2.73,0.0
1487,1488,The Garden Market,476661960,N/A Bev,Visa,Tax,2024-06-10 08:30:03,closed,df402438d0ea20b5,E HOWSER,e2sgardenman,4.92,0.64,0.0
1488,1489,The Garden Market,476665618,N/A Bev,Amex,Tax,2024-06-10 09:50:21,closed,2ae1a8983aa32643,VALUED CUSTOMER,e2sgardenman,8.19,0.00,0.0
1489,1490,The Garden Market,476673532,Beer,Discover,Tax,2024-06-10 11:29:54,closed,2ae1a8983aa32643,ANDREW SIMERMAN,e2sgardenman,15.30,1.39,0.0
1490,1491,The Garden Market,476696854,Beer,Visa,Tax,2024-06-10 14:03:10,closed,2ae1a8983aa32643,VISA CARDHOLDER,e2sgardenman,11.77,0.00,0.0


In [ ]:
# Drop duplicates:
location = location.drop_duplicates(subset=["Order ID"])
location.info()

<class 'pandas.core.frame.DataFrame'>
Index: 224 entries, 1486 to 8528
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              224 non-null    int64         
 1   Location                224 non-null    category      
 2   Order ID                224 non-null    int64         
 3   Reporting Group         224 non-null    category      
 4   Card Type               224 non-null    category      
 5   Display Name            224 non-null    category      
 6   Created At Time         224 non-null    datetime64[ns]
 7   State                   224 non-null    category      
 8   Terminal                224 non-null    category      
 9   Name                    224 non-null    object        
 10  Employee                224 non-null    category      
 11  Total                   224 non-null    float64       
 12  Total Credit Card Tips  224 non-null    float64    

In [ ]:
# checking duplicates
location.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
1486,1487,The Garden Market,476705704,N/A Bev,Visa,Tax,2024-06-10 15:12:30,closed,df402438d0ea20b5,VISA CARDHOLDER,e2sgardenman,30.01,2.73,0.0
1487,1488,The Garden Market,476661960,N/A Bev,Visa,Tax,2024-06-10 08:30:03,closed,df402438d0ea20b5,E HOWSER,e2sgardenman,4.92,0.64,0.0
1488,1489,The Garden Market,476665618,N/A Bev,Amex,Tax,2024-06-10 09:50:21,closed,2ae1a8983aa32643,VALUED CUSTOMER,e2sgardenman,8.19,0.00,0.0
1489,1490,The Garden Market,476673532,Beer,Discover,Tax,2024-06-10 11:29:54,closed,2ae1a8983aa32643,ANDREW SIMERMAN,e2sgardenman,15.30,1.39,0.0
1490,1491,The Garden Market,476696854,Beer,Visa,Tax,2024-06-10 14:03:10,closed,2ae1a8983aa32643,VISA CARDHOLDER,e2sgardenman,11.77,0.00,0.0


In [ ]:
# Make new columns:
def new_column(loc):
  loc["day"] = loc["Created At Time"].dt.day - 9
  loc["Shift AM/PM"] = loc["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
  loc["Shift Overlap"] = (loc["Created At Time"].dt.time >= pd.to_datetime('14:15:00').time()) & (loc["Created At Time"].dt.time <= pd.to_datetime('16:00:00').time())
  loc.tail()
  return loc
location = new_column(location)


<ipython-input-155-0af724e54b91>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc["day"] = loc["Created At Time"].dt.day - 9
<ipython-input-155-0af724e54b91>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc["Shift AM/PM"] = loc["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
<ipython-input-155-0af724e54b91>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [ ]:
def tables(loc):
  # Making a table for AM/PM shifts:
  loc_shifts_group = loc.groupby(["day", "Shift AM/PM"])["Total Credit Card Tips"].sum().reset_index(name="tips")
  table1 = loc_shifts_group.reset_index()

  # Making a table for Overlap shifts:
  loc_overlap_group = loc.groupby(["day", "Shift Overlap"])["Total Credit Card Tips"].sum().reset_index(name="tips")

  table2 = loc_overlap_group.loc[loc_overlap_group["Shift Overlap"] == True]

  # make table for agg:
  loc_agg_group = loc.groupby(["day"])["Total Credit Card Tips"].sum().reset_index(name="Total Tips")
  table3 = loc_agg_group
  return table1, table2, table3
table_set = tables(location)


In [ ]:
# Making a table for AM/PM shifts:
table_set[0]

,index,day,Shift AM/PM,tips
0,0,1,AM,7.14
1,1,1,PM,6.98
2,2,2,AM,84.34
3,3,2,PM,25.59
4,4,3,AM,150.38
5,5,3,PM,34.54


In [ ]:
# Making a table for Overlap shifts:
table_set[1]

,day,Shift Overlap,tips
1,1,True,6.98
3,2,True,18.61
5,3,True,34.54


In [ ]:
# Make table for agg:
table_set[2]

,day,Total Tips
0,1,14.12
1,2,109.93
2,3,184.92


# The Garden Tacos Nachos





In [ ]:
# Filter Amex Only:
garden_grill_tac_nach = transactions_df.loc[transactions_df["Location"].isin(["The Garden Tacos Nachos"])]
location = garden_grill_tac_nach
location.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
1512,1513,The Garden Tacos Nachos,476707328,Beer,Amex,Tax,2024-06-10 14:59:07,closed,Garden Tacos-4,VALUED CUSTOMER,e2sgardentaco,27.07,3.53,0.0
1513,1514,The Garden Tacos Nachos,476688242,Beer,Visa,Tax,2024-06-10 13:01:05,closed,Garden Tacos-4,LUIS GUTIERREZ,e2sgardentaco,49.42,4.49,0.0
1514,1515,The Garden Tacos Nachos,476697722,Beer,Visa,Tax,2024-06-10 14:08:06,closed,Garden Tacos-4,EVERETTE DOWNS,e2sgardentaco,53.56,4.87,0.0
1515,1516,The Garden Tacos Nachos,476661284,Food,Visa,Tax,2024-06-10 07:42:12,closed,G,VISA CARDHOLDER,e2sgardentaco,26.51,4.04,0.0
1516,1517,The Garden Tacos Nachos,476676991,N/A Bev,Visa,Tax,2024-06-10 11:52:11,closed,Garden Tacos-3,RUSSELL BERRY,e2sgardentaco,36.37,0.00,0.0


In [ ]:
# Drop duplicates:
location = location.drop_duplicates(subset=["Order ID"])
location.info()

<class 'pandas.core.frame.DataFrame'>
Index: 976 entries, 1512 to 9338
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              976 non-null    int64         
 1   Location                976 non-null    category      
 2   Order ID                976 non-null    int64         
 3   Reporting Group         976 non-null    category      
 4   Card Type               976 non-null    category      
 5   Display Name            972 non-null    category      
 6   Created At Time         976 non-null    datetime64[ns]
 7   State                   976 non-null    category      
 8   Terminal                976 non-null    category      
 9   Name                    976 non-null    object        
 10  Employee                976 non-null    category      
 11  Total                   976 non-null    float64       
 12  Total Credit Card Tips  976 non-null    float64    

In [ ]:
# checking duplicates
location.head()

,Unnamed: 0,Location,Order ID,Reporting Group,Card Type,Display Name,Created At Time,State,Terminal,Name,Employee,Total,Total Credit Card Tips,Order Discounts
1512,1513,The Garden Tacos Nachos,476707328,Beer,Amex,Tax,2024-06-10 14:59:07,closed,Garden Tacos-4,VALUED CUSTOMER,e2sgardentaco,27.07,3.53,0.0
1513,1514,The Garden Tacos Nachos,476688242,Beer,Visa,Tax,2024-06-10 13:01:05,closed,Garden Tacos-4,LUIS GUTIERREZ,e2sgardentaco,49.42,4.49,0.0
1514,1515,The Garden Tacos Nachos,476697722,Beer,Visa,Tax,2024-06-10 14:08:06,closed,Garden Tacos-4,EVERETTE DOWNS,e2sgardentaco,53.56,4.87,0.0
1515,1516,The Garden Tacos Nachos,476661284,Food,Visa,Tax,2024-06-10 07:42:12,closed,G,VISA CARDHOLDER,e2sgardentaco,26.51,4.04,0.0
1516,1517,The Garden Tacos Nachos,476676991,N/A Bev,Visa,Tax,2024-06-10 11:52:11,closed,Garden Tacos-3,RUSSELL BERRY,e2sgardentaco,36.37,0.00,0.0


In [ ]:
# Make new columns:
def new_column(loc):
  loc["day"] = loc["Created At Time"].dt.day - 9
  loc["Shift AM/PM"] = loc["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
  loc["Shift Overlap"] = (loc["Created At Time"].dt.time >= pd.to_datetime('14:15:00').time()) & (loc["Created At Time"].dt.time <= pd.to_datetime('16:00:00').time())
  loc.tail()
  return loc
location = new_column(location)


<ipython-input-163-0af724e54b91>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc["day"] = loc["Created At Time"].dt.day - 9
<ipython-input-163-0af724e54b91>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc["Shift AM/PM"] = loc["Created At Time"].dt.time.apply(lambda x: "AM" if x <= pd.to_datetime('14:15:00').time() else "PM")
<ipython-input-163-0af724e54b91>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [ ]:
def tables(loc):
  # Making a table for AM/PM shifts:
  loc_shifts_group = loc.groupby(["day", "Shift AM/PM"])["Total Credit Card Tips"].sum().reset_index(name="tips")
  table1 = loc_shifts_group.reset_index()

  # Making a table for Overlap shifts:
  loc_overlap_group = loc.groupby(["day", "Shift Overlap"])["Total Credit Card Tips"].sum().reset_index(name="tips")

  table2 = loc_overlap_group.loc[loc_overlap_group["Shift Overlap"] == True]

  # make table for agg:
  loc_agg_group = loc.groupby(["day"])["Total Credit Card Tips"].sum().reset_index(name="Total Tips")
  table3 = loc_agg_group
  return table1, table2, table3
table_set = tables(location)


In [ ]:
# Making a table for AM/PM shifts:
table_set[0]

,index,day,Shift AM/PM,tips
0,0,1,AM,264.63
1,1,1,PM,110.68
2,2,2,AM,466.33
3,3,2,PM,84.85
4,4,3,AM,568.42
5,5,3,PM,127.28


In [ ]:
# Making a table for Overlap shifts:
table_set[1]

,day,Shift Overlap,tips
1,1,True,78.25
3,2,True,60.38
5,3,True,103.64


In [ ]:
# Make table for agg:
table_set[2]

,day,Total Tips
0,1,375.31
1,2,551.18
2,3,695.70
